# Script in R to produce the standardized catalog

### Adding a R kernel for Jupyter 

In a linux window, type the command:
> conda install -c r ipython-notebook r-irkernel


In [93]:
# Reading of resulting Xmatch catalog (radius Xmatch 3 arcsec, Xmatch on Metadata _RAJ2000 & _DEJ2000) 
#Don <-read.csv("/Users/joncouri/Desktop/Student_UMD/Catalogs/Tgas200_Wright2003_2MASS_FinCat.csv")

Don <-read.csv("/Users/joncouri/Git_GAIA/GAIA/Work-Areas/ijoncour/StandCat/Tgas200_Wright2003_2MASS_FinCat.csv")
length(Don$Teff) # Number of stars 75753


[1] 75753

In [86]:
# Check on Don
str(Don)
# Check on type 
class(Don)

'data.frame':	75753 obs. of  51 variables:
 $ angDist    : num  0.1526 0.0318 0.0925 0.2092 0.1613 ...
 $ d_arcsec   : num  0.02561 0.00417 0.0382 0.05663 0.0232 ...
 $ X_RAJ2000  : num  115.7 33.7 164.8 45.5 170.7 ...
 $ X_DEJ2000  : num  76.6 21.4 -21.7 -25.5 -34.1 ...
 $ HIP        : int  NA NA NA 14122 55548 NA NA NA 74271 NA ...
 $ TYC2       : Factor w/ 41885 levels "","1-1122-1",..: 16622 1228 25106 1 1 11053 34280 13550 1 41786 ...
 $ Source     : num  1.14e+18 9.96e+16 3.55e+18 5.07e+18 5.40e+18 ...
 $ RA_ICRS    : num  115.7 33.7 164.8 45.5 170.7 ...
 $ e_RA_ICRS  : num  0.188 0.676 0.254 0.181 0.206 0.286 0.274 0.301 0.228 0.461 ...
 $ DE_ICRS    : num  76.6 21.4 -21.7 -25.5 -34.1 ...
 $ e_DE_ICRS  : num  0.301 0.237 0.138 0.173 0.145 0.173 0.265 0.268 0.16 0.528 ...
 $ Plx        : num  10.23 5.65 10.05 10.2 8.26 ...
 $ e_Plx      : num  0.25 0.25 0.37 0.24 0.29 0.29 0.28 0.38 0.25 0.25 ...
 $ pmRA       : num  -20 -5 -92.1 112.5 102.5 ...
 $ e_pmRA     : num  0.437 1.316 1

[1] "data.frame"

In [94]:
# Copying Don in another variable
Data <- Don

# Filtering Dup=1 stars (multiples or visual stars as flagged by GAIA )
 testind_d <- which(Data$Dup ==0)
Data_Dup <- Data[testind_d ,]
 
# Copying the filtered dataframe free of sources having Dup=1  in dataframe Dat1
Dat1 <- Data_Dup # Number of stars at the stage  50324
length(Dat1$Teff)

[1] 50324

In [95]:
# FILTER Get rid of duplicated sources  from GAIA ID (column name: Source) - NUMBER FILTERED SOURCES: 190 Sources
Name <- Dat1$Source

# Procedure to filter the second occurence of individual Source and keep the first one
    #testdep<- duplicated(Name)
    # testdep_False <-which(testdep==FALSE)
    #Data_Dup_depfree <- Data_DistLim[testdep_False,]

# Procedure the filter all the sources that appear multiple times as GAIA sources
    testdep_one <- !(duplicated(Name) | duplicated(Name, fromLast = TRUE)) 
    testdep_one_TRUE <- which(testdep_one==TRUE)
     Data_Dup_depfree <- Data_Dup[testdep_one_TRUE,]
 
# Copying the filtered dataframe free of all duplicated sources in dataframe Dat2
Dat2 <-  Data_Dup_depfree  # Number of stars 50134 stars
length(Dat2$Teff)


[1] 50134

In [96]:
# FILTER Get rid of duplicated sources  from 2MASS ID (column name: X2MASS) - NUMBER FILTERED SOURCES: 18 Sources
Name2 <- as.character(Dat2$X2MASS)

# Procedure the filter all the sources that appear multiple times as 2MASS sources
    testdep_two <- !(duplicated(Name2) | duplicated(Name2, fromLast = TRUE)) 
    testdep_two_TRUE <- which(testdep_two==TRUE)
     Data_Dup_depfree2 <- Dat2[testdep_two_TRUE,]

# Copying the filtered dataframe free of all duplicated sources in dataframe Dat3
Dat3 <-  Data_Dup_depfree2  # Number of remaining stars 50116  stars
length(Dat3$Teff)

[1] 50116

In [97]:
#-------------- MASK Operations on the "goodness" of flags (Qfl) in 2MASS

    Nb_stars <- length(Dat3$Teff)

# Mask 2MASS photometric magnitudes of sources for which the flags  are not "AAA" 
#"AAA" means (detection at 10 sigmas in each JHK bands)
    
    # Ind_JHK_ok <- which(substr(as.character(Dat3$Qfl), 1, 1) == "A" &substr(as.character(Dat3$Qfl), 2, 2) == "A" 
    #                        & substr(as.character(Dat3$Qfl), 3, 3) == "A"  )
    # Ind_JHK_nok <- seq(1,Nb_stars)[-Ind_JHK_ok]
    # Dat3$Jmag[Ind_JHK_nok] <- NA
    # Dat3$Hmag[Ind_JHK_nok] <- NA
    # Dat3$Kmag[Ind_JHK_nok] <- NA



# Mask each 2MASS photometric magnitude that does not have a flag "A" in the band band 
    Ind_J_ok <- which(substr(as.character(Dat3$Qfl), 1, 1) == "A" )
    Ind_H_ok <- which(substr(as.character(Dat3$Qfl), 2, 2) == "A" )
    Ind_K_ok <- which(substr(as.character(Dat3$Qfl), 3, 3) == "A" )

    Ind_J_nok <- seq(1,Nb_stars)[-Ind_J_ok] # 1847  sources masked in J
    Ind_H_nok <- seq(1,Nb_stars)[-Ind_H_ok] # 2057 sources masked in H
    Ind_K_nok <- seq(1,Nb_stars)[-Ind_K_ok] # 1518  sources masked in K
    length(Ind_J_nok)
    length(Ind_H_nok)
    length(Ind_K_nok)

    Dat3$Jmag[Ind_J_nok] <- NA
    Dat3$Hmag[Ind_H_nok] <- NA
    Dat3$Kmag[Ind_K_nok] <- NA

# Check on the number of sources that are masked simultaneously in the 3 bands : 1071 sources 
    Ind_JHK_na <-which(is.na(Dat3$Jmag) & is.na(Dat3$Hmag)&is.na(Dat3$Kmag) ==TRUE)
    Ind_JHK_non_na <- seq(1,Nb_stars)[-Ind_JHK_na] #
    length(Ind_JHK_na) 

length(Dat3$Teff)

[1] 1847

[1] 2057

[1] 1518

[1] 1071

[1] 50116

In [98]:
# LAST FILTER 
## Check and keep only sources that have Xmatch distances smaller than 1.5 arcsec => 50093 final sources
# require both less than 1.5 arcsec (23 sources are removed)
DistLim <- 1.5
 testind_d <- which(Dat3$angDist <=DistLim  & Dat3$d_arcsec <= DistLim)
length(testind_d)
Data_Dup_depfree2_distlim <- Dat3[testind_d ,]

# Copying the filtered dataframe free of Xmatch [distance > 1.5 arcsec] sources in dataframe Dat
Dat4 <-  Data_Dup_depfree2_distlim  # Number of remaining stars 50116  stars
length(Dat4$Teff)

[1] 50093

[1] 50093

In [99]:

## MASKED GMAG SUCH THAT ABS( G-J - MEAN(G-J))/SDT  <=5 ==> 1860 sources have the Gmag value masked 

    Nb_stars <- length(Dat4$Teff)
 moy_Delta_G_J <- mean(Dat4$Jmag-Dat4$X.Gmag.,na.rm = TRUE)
 sd_Delta_G_J <- sd(Dat4$Jmag-Dat4$X.Gmag.,na.rm = TRUE)
 test_Diff_G_J <- ((Dat4$Jmag-Dat4$X.Gmag.)-moy_Delta_G_J)/sd_Delta_G_J 
 Ind_Diff_G_J_ok <- which(abs(test_Diff_G_J )<=5)
 Ind_Diff_G_J_nok <- seq(1, Nb_stars)[-Ind_Diff_G_J_ok]

Dat4$X.Gmag.[ Ind_Diff_G_J_nok ] <- NA
 
length(Ind_Diff_G_J_nok)

[1] 1860

In [100]:
# Final dataframe
FinalDat <- Dat4

In [102]:
# Writing The output catalog
#filout = "/Users/joncouri/Git_GAIA/GAIA/Work-Areas/ijoncour/StandCat/Tgas200_Wright2003_2MASS_FinCat_out.csv"
Filout <- "output.csv"
write.csv(FinalDat, file =Filout )